# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
spaceship = spaceship.dropna(how='any', axis=0, )

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
spaceship['Cabin'] = spaceship['Cabin'].str[0]

spaceship['Cabin'] = spaceship['Cabin'].replace('T', 'A')

print(spaceship['Cabin'].head())

0    B
1    F
2    A
3    A
4    F
Name: Cabin, dtype: object


In [5]:
spaceship.drop(columns=['PassengerId', 'Name', 'ShoppingMall'], inplace=True)
encoded_df = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep','Cabin', 'Destination', 'VIP'], dtype=int)

In [6]:
features = encoded_df.drop(columns=['Transported']) #X (x1, x2, ..., xp)

target = encoded_df['Transported'] #y

In [7]:
X_train, X_test, y_train, y_test = train_test_split (features, target, test_size=0.2, random_state=12)

normalizer = MinMaxScaler()

normalizer.fit(X_train)

X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)


In [8]:
#your code here

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [9]:
#your code here
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)
bagging_reg.fit(X_train, y_train)

pred = bagging_reg.predict(X_test)


- Evaluate your model

In [10]:
#your code here
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

MAE 0.2807976038299915
RMSE 0.37571331285581483
R2 score 0.43535802617163444


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [17]:
param_grid = {
    'estimator__max_depth': [10, 20],
    'estimator__min_samples_split': [2, 5],
    'estimator__min_samples_leaf': [1, 2],
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0]
}

#your code here

- Run Grid Search

In [18]:
# Apply Grid Search

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score

grid_search = GridSearchCV(estimator=bagging_reg, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters from Grid Search
print("Best parameters found by Grid Search:", grid_search.best_params_)



Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters found by Grid Search: {'estimator__max_depth': 10, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'max_features': 1.0, 'max_samples': 0.5}


- Evaluate your model